In [1]:
%reload_ext autoreload 
%autoreload 2


In [2]:
import requests 

In [3]:
import src.config as config

In [4]:
import hopsworks

In [5]:
project = hopsworks.login(
    project=config.HOPSWORKS_PROJECT_NAME,
    api_key_value=config.HOPSWORKS_API_KEY
)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/845972


In [14]:
feature_store = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.


In [7]:
from src.data import load_and_validate_data, transform_raw_data_into_ts_data, get_cutoff_indices, transform_ts_data_to_features_and_target

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/845972


In [8]:
url = 'https://www.banrep.gov.co/sites/default/files/Serie_historica_ipvnbr.xlsx'

In [9]:
response = requests.get(url)
print(response.status_code)

200


In [10]:
df = load_and_validate_data(response)

<Response [200]>
2024-07-08 22:33:53,114 WARNING: FutureWarning: Passing bytes to 'read_excel' is deprecated and will be removed in a future version. To read from a byte string, wrap it in a `BytesIO` object.



In [11]:
df_list = transform_raw_data_into_ts_data(df)

### Bogota


### time series

In [12]:
df_bogota = df_list[0]

In [15]:
ts_group = feature_store.get_or_create_feature_group(
                                                            name= config.FEATURE_GROUP_NAME_TS,
                                                            version= config.FEATURE_GROUP_VERSION,
                                                            description= 'Time series bogota',
                                                            primary_key= ['fecha']
)

In [16]:
ts_group.insert(df_bogota, write_options={'wait_for_job':False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/845972/fs/841795/fg/983397


Uploading Dataframe: 0.00% |          | Rows 0/221 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: time_series_24_months_bogota_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/845972/jobs/named/time_series_24_months_bogota_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x1f5aaaec190>, None)

In [17]:
try:
    #create feature view
    feature_store.create_feature_view(
        name=config.FEATURE_VIEW_NAME_TS,
        version=config.FEATURE_VIEW_VERSION,
        query=ts_group.select_all()
    )
    
except:
    print('Feature view already existed. Skip creation')
    
#get feature view
fature_view = feature_store.get_feature_view(
    name= config.FEATURE_VIEW_NAME_TS,
    version= config.FEATURE_VIEW_VERSION
)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/845972/fs/841795/fv/time_series_24_months_bogota/version/2


### features an target

In [18]:
df_ft = transform_ts_data_to_features_and_target(df_bogota, 'bogota')

0it [00:00, ?it/s]

2024-07-08 22:35:30,216 WARNING: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)



208it [00:00, 1341.94it/s]


In [19]:
feature_group = feature_store.get_or_create_feature_group(
                                                            name= config.FEATURE_GROUP_NAME,
                                                            version= config.FEATURE_GROUP_VERSION ,
                                                            description= 'features and target bogota',
                                                            primary_key= ['Target']
)


In [20]:
# Exportar los datos al archivo train
feature_group.insert(df_ft, write_options={'wait_for_job':False})


2024-07-08 22:35:44,513 WARNING: FeatureGroupWarning: The ingested dataframe contains upper case letters in feature names: `['Target']`. Feature names are sanitized to lower case in the feature store.

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/845972/fs/841795/fg/981356


Uploading Dataframe: 0.00% |          | Rows 0/208 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: features_target_monthly_bogota_2_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/845972/jobs/named/features_target_monthly_bogota_2_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x1f5aaafe750>, None)

In [21]:
# create feature view
# this feature view is only uses on feature group, so the query is trivial

try:
    #create feature view
    feature_store.create_feature_view(
        name=config.FEATURE_VIEW_NAME,
        version=config.FEATURE_VIEW_VERSION,
        query=feature_group.select_all()
    )
    
except:
    print('Feature view already existed. Skip creation')
    
#get feature view
fature_view = feature_store.get_feature_view(
    name= config.FEATURE_VIEW_NAME,
    version= config.FEATURE_VIEW_VERSION
)



Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/845972/fs/841795/fv/features_target_monthly_bogota/version/2
